# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Laboratorio 3: Deep Learning

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169

**Curso:** Data Science  
**Sección:** 10  

---

Guatemala, 08 de agosto de 2024


<div style="text-align: center;">
    <h1>✍️ Discusión / Cálculos - [Deep Learning] ✍️</h1>
</div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv
import statsmodels.api as sm
import matplotlib.pyplot as plt
from autoviz.AutoViz_Class import AutoViz_Class
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
%matplotlib inline

c:\Users\lijv1.LINDA_HP\Documents\GitHub\Lab03_Data-Science\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


Imported v0.1.905. Please call AutoViz in this sequence:
    AV = AutoViz_Class()
    %matplotlib inline
    dfte = AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=1, lowess=False,
               chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30, save_plot_dir=None)


In [3]:
csvPath = 'Consumo/dataConsumo.csv'
dataConsumo = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

csvPath = 'Importacion/dataImportacion.csv'
dataImportacion = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

csvPath = 'PreciosPromedioNacionales/dataPrecios.csv'
dataPrecios = pd.read_csv(csvPath)
print(f"✅ Archivo cargado: {csvPath}")

✅ Archivo cargado: Consumo/dataConsumo.csv
✅ Archivo cargado: Importacion/dataImportacion.csv
✅ Archivo cargado: PreciosPromedioNacionales/dataPrecios.csv


In [4]:
serieConsumoDiesel = dataConsumo[['Fecha', 'Diesel']]
serieImportacionRegular = dataImportacion[['Fecha', 'Gasolina regular']]
seriePreciosSuper = dataPrecios[['FECHA', 'Superior GTQ/GALON']]
serieConsumoDiesel.set_index('Fecha', inplace=True)
serieConsumoDiesel.sort_index(inplace=True)
serieImportacionRegular.set_index('Fecha', inplace=True)
serieImportacionRegular.sort_index(inplace=True)
seriePreciosSuper.set_index('FECHA', inplace=True)
seriePreciosSuper.sort_index(inplace=True)

### 🧪 Division train y test

In [5]:
# Dividir los datos en 70% entrenamiento y 30% prueba

# Diesel
n = len(serieConsumoDiesel)
train_size = int(n * 0.7)
train, test = serieConsumoDiesel[:train_size], serieConsumoDiesel[train_size:]

# Gasolina Regular
n = len(serieImportacionRegular)
train_size = int(n * 0.7)
train2, test2 = serieImportacionRegular[:train_size], serieImportacionRegular[train_size:]

# Gasolina Superior
n = len(seriePreciosSuper)
train_size = int(n * 0.7)
train3, test3 = seriePreciosSuper[:train_size], seriePreciosSuper[train_size:]